# The Code
To calculate a random set of coordinates for a possible drawing of a Kontsevich' graph.

In [1]:
def b_vector(encoding,delta,inclines):
     #saved under the name 'b_vector_algorithm'
     r"""
     Return the vector $b$ in the system $Ax=b$
     
     INPUT:
     
     - ''encoding'' - a vector of the form $[m,n,s,e_1,e_2,...,e_2n]$
     
     - ''delta'' - a positive scalar
     
     - ''inclines'' - a matrix of 2n inclines, of the form $[a_1^L,b_1^L; ... ;a_n^R,b_n^R]$
     
     OUTPUT:
     
     - a vector $b$ of size 2n x 1
     
     EXAMPLES:
     
         sage: b_vector(vector([2,1,1,0,1]),1,matrix([[1,1],[-1,1]]))
         (0,1)
         sage: b_vector(vector([2,2,1,0,1,1,2]),1,\matrix([[0,-1],[1,-1],[-1,0],[0,-1]]))
         (0,-1,0,0)
     """
     m=encoding[0]
     n=encoding[1]
    
     b=zero_vector(2*n)
     for i in range(2*n):
         e=encoding[i+3]
         if e < m:
             b[i]=e*delta*inclines[i,1]
     return b

In [2]:
def A_matrix(encoding,inclines):
     #saved under the name A_matrix_algorithm
     r"""
     Return the matrix A in the system Ax=b
     
     INPUT:
     
     - ''encoding'' - a vector of the form (m,n,s,e_1,e_2,...,e_2n)
     
     - ''inclines'' - a 2n x 2 matrix of the form [a_1^L,b_1^L;...;a_n^R,b_n^R]
     
     OUTPUT:
     
     - a matrix A of size 2n x 2n
     
     EXAMPLES:
         sage: A_matrix(vector([2,1,1,0,1]),matrix([[1 ,1] ,[ -1 ,1]]))
         [1,-1]
         [1,1]
         
         sage: A_matrix(vector([2,2,1,0,1,1,2]),matrix([[0,-1],[1,-1],[-1,0],[0,-1]]))
         [-1  0  0  0]
         [-1 -1  0  0]
         [ 0  0  0  1]
         [ 1  0 -1  0]
     
     """
     m=encoding[0]
     n=encoding[1]
     A=zero_matrix(2*n,2*n)
     for i in range(2*n):
         e=encoding[i+3]
         if i%2 == 0:
             A[i,i]=inclines[i,1]
             A[i,i+1]=-inclines[i,0]
             if e >= m:
                 A[i,2*(e-m+1)-2]=-inclines[i,1]
                 A[i,2*(e-m+1)-1]=inclines[i,0]
         if i%2 == 1:
             A[i,i]=-inclines[i,0]
             A[i,i-1]=inclines[i,1]
             if e >= m:
                 A[i,2*(e-m+1)-2]=-inclines[i,1]
                 A[i,2*(e-m+1)-1]=inclines[i,0]
     return A

In [3]:
def inclines_to_coordinates(encoding,inclines,delta):
     #saved under the name 'inclines_to_coordinates'
        
     r"""
     returns a vector x containing the coordinates of the system defined by the encoding, inclines and delta
     
     INPUT:
     
     - ''encoding'' - a vector of the form $[m,n,s,e_1,e_2,...,e_2n]$
     
     - ''inclines'' - a matrix of 2n inclines, of the form $[a_1^L,b_1^L; ... ;a_n^R,b_n^R]$
     
     - ''delta'' - a positive scalar
     
     OUTPUT:
     
     - a 2n x 1 vector x containing the coordinates of the form (x_1,y_1,...,x_n,y_n)
     
     EXAMPLES:
         sage: inclines_to_coordinates(vector([2,1,1,0,1]),matrix([[1 ,1] ,[ -1 ,1]]),1)
         (1/2,1/2)
         sage: inclines_to_coordinates(vector([2,2,1,0,1,0,1]),matrix([[0,1],[1,-1],[1,1],[0,1]]),1)
         (0,1,1,1)
     
     """
     A=A_matrix(encoding,inclines)
     b=b_vector(encoding,delta,inclines)
    
     if det(A)==0:
         x=0
     if abs(det(A))>0:
         x=A\b
     return x

In [4]:
def DrawGraph_algorithm(encoding,delta):
     #saved under the name 'DrawGraph_algorithm'
     r"""
     returns a matrix with coordinates of the internal nodes
     
     INPUT: 
     
     - ''encoding'' - a vector of the form $[m,n,s,e_1,e_2,...,e_2n]$
     
     - ''delta'' - a positive scalar
     
     OUTPUT:
     
     - an n x 2 matrix containing coordinates in the form [x_1,y_1;...;x_n,y_n],
         because of randomization, the coordinates can change with each use of the function 
     
     EXAMPLES:
         sage: DrawGraph_algorithm(vector([2,1,1,0,1]),1)
         [ 0.818181818181818 -0.272727272727273]
         sage: DrawGraph_algorithm(vector([2,2,1,0,1,0,1]),1)
         [ 3.00000000000000  3.00000000000000]
         [0.307692307692308 0.230769230769231]
                  
     """

     #we're first generating all possible inclines, stored in set_of_inclines
     positive_inclines=matrix([[0,1],[1,0],[1,1],[1,2],[1,3],[1,4],\
     [2,1],[2,3],[3,1],[3,2],[3,4],[4,1],[4,3]])
     nofinclines=(positive_inclines.nrows()-1)*2
     half_nofi=nofinclines/2
     set_of_inclines=zero_matrix(nofinclines,2)
     for i in range(nofinclines):
         if i<=half_nofi:
             set_of_inclines[i,:]=positive_inclines[i,:]
         if i>half_nofi:
             set_of_inclines[i,0]=positive_inclines[i-half_nofi+1,0]
             set_of_inclines[i,1]=-positive_inclines[i-half_nofi+1,1]

     #from here on we will be computing the solution x
     #for a random set of inclines
     n=encoding[1]
     inclines=zero_matrix(2*n,2)
     x=0
     while x==0:
         for i in range(2*n):
             k=ZZ.random_element(0,nofinclines)
             inclines[i,:]=set_of_inclines[k,:]
             if i%2 == 1:
                 if inclines[i,:]==inclines[i-1,:]:
                     if k==nofinclines-1:
                         inclines[i,:]=set_of_inclines[0,:]
                     if k<nofinclines-1:
                         inclines[i,:]=set_of_inclines[k+1,:]
         x=inclines_to_coordinates(encoding,inclines,delta)
                    
     #changing the look of the output
     coordinates=zero_matrix(RR,n,2)
     for i in range(n):
         coordinates[i,0]=x[2*i]
         coordinates[i,1]=x[2*i+1]
        
     return coordinates

In [5]:
def positive_test(coordinates):
    
    r""""
    returns a value True or False based on whether the coordinates all lie above the line y=0
    
    INPUT:
    
    - ''coordinates'' - an n x 2 matrix containing coordinates in the form [x_1,y_1;...;x_n,y_n]
    
    OUTPUT:
    
    - a value False or True, indicating whether the input matrix had coordinates below or on the line y=0
    
    EXAMPLES:
        sage: positive_test(matrix([[1,4],[2,3]]))
        True

        sage: positive_test(matrix([[1,4],[2,-2]]))
        False
        
        sage: positive_test(matrix([[1,0],[2,3],[4,5]]))
        False
    
    """
    i=0
    n=coordinates.nrows()
    value=True
    while i<n:
         if coordinates[i,1]<=0:
             value = False
             i=n
         i=i+1
    return value

In [6]:
def same_vertices_test(coordinates):
    
    r""""
    returns a value True or False indicating whether the coordinates all indicate unique points
    
    INPUT:
    
    - ''coordinates'' - an n x 2 matrix containing coordinates in the form [x_1,y_1;...;x_n,y_n]
    
    OUTPUT:
    
    - a value False or True, indicating whether the input matrix has unique coordinates
    
    EXAMPLES:
        sage: same_vertices_test(matrix([[1,2],[3,4],[1,2]]))
        False
        sage: same_vertices_test(matrix([[1,2],[3,4],[1,3]]))
        True
    
    """
    n=coordinates.nrows()
    value=True
    for i in range(n):
        for j in range(n):
            if coordinates[i,:]==coordinates[j,:]:
                if i!=j:
                    value = False
    return value 

In [7]:
def length_test(encoding,delta,coordinates,min_len):
    r"""
    returns a value False or True 
    depending on whether any of the edges defined by the coordinates and encoding has 
    a length smaller than the minimal value min_len
    
    INPUT:
    
     - ''encoding'' - a vector of the form $[m,n,s,e_1,e_2,...,e_2n]$
     
     - ''delta'' - a positive scalar
     
     - ''coordinates'' - an n x 2 matrix containing coordinates in the form [x_1,y_1;...;x_n,y_n]
     
     - ''min_len'' - a positive scalar
     
    OUTPUT:
    
    - a value False or True, denoting whether all edges of the defined graph are larger than the minimum lenght
    
    EXAMPLES:
        sage: length_test(vector([2,3,1,0,1,0,4,1,2]),40,matrix([[160,40],[210,105],[280,0]]),10)
        True
        sage: length_test(vector([2,1,1,0,1]),10,matrix([[1,3]]),10)
        False

    """
    
    
    m=encoding[0]
    n=encoding[1]
    value=True
    
    sink_coord=zero_matrix(RR,m,2)
    for i in range(m):
        sink_coord[i,0]=i*delta
    
    for i in range(n):
        source_vert_x=coordinates[i,0]
        source_vert_y=coordinates[i,1]
        
        
        goal_vert_L=encoding[1+2*i]
        goal_vert_R=encoding[2+2*i]
        if goal_vert_L <m:
            goal_vert_L_x=sink_coord[goal_vert_L,0]
            goal_vert_L_y=sink_coord[goal_vert_L,1]
        if goal_vert_L>=m:
            goal_vert_L_x=coordinates[goal_vert_L-m,0]
            goal_vert_L_y=coordinates[goal_vert_L-m,1]
            
        if goal_vert_R <m:
            goal_vert_R_x=sink_coord[goal_vert_R,0]
            goal_vert_R_y=sink_coord[goal_vert_R,1]
        if goal_vert_R>=m:
            goal_vert_R_x=coordinates[goal_vert_R-m,0]
            goal_vert_R_y=coordinates[goal_vert_R-m,1]    
        
        delta_L_x=source_vert_x-goal_vert_L_x
        delta_L_y=source_vert_y-goal_vert_L_y
        delta_R_x=source_vert_x-goal_vert_R_x
        delta_R_y=source_vert_y-goal_vert_R_y
        
        if delta_L_x!=0:
            if delta_L_y!=0:
                edge_length_sq=delta_L_x^2+delta_L_y^2
                edge_length=sqrt(edge_length_sq)
                if edge_length<min_len:
                    value=False
                    
        if delta_R_x!=0:
            if delta_R_y!=0:
                edge_length_sq=delta_R_x^2+delta_R_y^2
                edge_length=sqrt(edge_length_sq)
                if edge_length<min_len:
                    value=False    
    
    
    return value

In [8]:
def overlapping_edges_test(encoding,delta,n_coordinates):
    r"""
    returns a value False or True, depending on whether the defined graph has overlapping edges
    
    INPUT:
    
     - ''encoding'' - a vector of the form $[m,n,s,e_1,e_2,...,e_2n]$
     
     - ''delta'' - a positive scalar
     
     - ''n_coordinates'' - an n x 2 matrix containing coordinates in the form [x_1,y_1;...;x_n,y_n]
     
     OUTPUT:
     
     - a value False or True,  depending on whether the defined graph has overlapping edges
     
     EXAMPLES:
         sage: overlapping_edges_test(vector([2,2,1,0,1,1,2]),10,matrix([[0,10],[5,5]]))
         False
         sage: overlapping_edges_test(vector([2,2,1,0,1,1,2]),10,matrix([[0,10],[10,10]]))
         True
    
    """
    m=encoding[0]
    n=encoding[1]
    value=True
    
    sink_coord=zero_matrix(RR,m,2)
    for i in range(m):
        sink_coord[i,0]=i*delta
    
    coordinates=block_matrix([[sink_coord],[n_coordinates]])

    edges_source_goal=zero_matrix(QQ,2*n,4)
    for i in range(2*n):
        if i%2==0:
            source_vertice_label=i/2+m
        else:
            source_vertice_label=(i-1)/2+m
        source_vertice_x=coordinates[source_vertice_label,0]
        source_vertice_y=coordinates[source_vertice_label,1]

        goal_vertice_label=encoding[3+i]
        goal_vertice_x=coordinates[goal_vertice_label,0]
        goal_vertice_y=coordinates[goal_vertice_label,1]

        edges_source_goal[i,:]=vector([source_vertice_x,source_vertice_y,goal_vertice_x,goal_vertice_y])
    
    for i in range(2*n):
        p_0=vector([edges_source_goal[i,0],edges_source_goal[i,1]])
        p_1=vector([edges_source_goal[i,2],edges_source_goal[i,3]])
        if p_0==p_1:
            continue
        r=p_1-p_0
        
        for j in range(i+1,2*n):
            q_0=vector([edges_source_goal[j,0],edges_source_goal[j,1]])
            q_1=vector([edges_source_goal[j,2],edges_source_goal[j,3]])
            if q_0==q_1:
                continue
            
            if p_0 == q_1 and p_1==q_0:
                continue

            s=q_1-q_0
            r_cross_s=r[0]*s[1]-r[1]*s[0]

            q_min_p=q_0-p_0
            q_min_p_cross_r=q_min_p[0]*r[1]-q_min_p[1]*r[0]

            if r_cross_s == 0 and q_min_p_cross_r == 0:
                t_0=q_min_p*r/(r*r)
                t_1=t_0+s*r/(r*r)
                if 0<=t_0<=1 or 0<=t_1<=1:
                    value = False
                    break

    return value

In [9]:
def DrawGraph_filter(encoding,delta,min_len):
    
    r"""
    returns coordinates that are a proper solution to the system
    
    INPUT: 
     
     - ''encoding'' - a vector of the form $[m,n,s,e_1,e_2,...,e_2n]$
     
     - ''delta'' - a positive scalar
     
     - ''min_len'' - a positive scalar
     
     OUTPUT:
     
     - an n x 2 matrix containing coordinates in the form [x_1,y_1;...;x_n,y_n],
         because of randomization, the coordinates can change with each use of the function 
         
     EXAMPLES:
         sage: DrawGraph_filter(vector([2,1,1,0,1]),40,10)
         [120.000000000000 240.000000000000]
         
         sage: DrawGraph_filter(vector([2,2,1,0,1,0,1]),40,10)
         [-32.0000000000000  96.0000000000000]
         [ 48.0000000000000  16.0000000000000]
         
    """
    value = False
    k=0
    while value == False:
        coordinates=DrawGraph_algorithm(encoding,delta)
        v1=positive_test(coordinates)
        v2=same_vertices_test(coordinates)
        v3=length_test(encoding,delta,coordinates,min_len)
        v4=overlapping_edges_test(encoding,delta,coordinates)
        value = all([v1,v2,v3,v4])
        k=k+1
        if k == 10^3:
            coordinates=0
            break
        
    return coordinates

In [10]:
def DrawGraph_compute_and_draw(encoding,delta,min_len):
    r"""
    returns a randomized plot of the graph defined by the encoding
    
    INPUT: 
     
     - ''encoding'' - a vector of the form $[m,n,s,e_1,e_2,...,e_2n]$
     
     - ''delta'' - a positive scalar
     
     - ''min_len'' - a positive scalar
     
     OUTPUT:
     
     - a graphics plot 
         
    """
    
    m=encoding[0]
    n=encoding[1]
    
    n_coordinates=DrawGraph_filter(encoding,delta,min_len)
    m_coordinates=zero_matrix(RR,m,2)
    
    for i in range(m):
        m_coordinates[i,0]=delta*i
        
    coord=block_matrix([[m_coordinates],[n_coordinates]])
    coordinates=coord.rows()
    points=point2d(coord)
    plot_list=[points]
    
    for i in range(n):
        source_vertice=coordinates[m+i]
        
        left_vertice_label=encoding[2*i+3]
        right_vertice_label=encoding[2*i+4]
        
        left_vertice=coordinates[left_vertice_label]
        right_vertice=coordinates[right_vertice_label]
        
        left_edge=arrow2d(source_vertice,left_vertice)
        right_edge=arrow2d(source_vertice,right_vertice)
        
        plot_list.append(left_edge)
        plot_list.append(right_edge)
        
        plot=points
    for i in range(len(plot_list)-1):
        plot+=plot_list[i+1]
    
    
    return plot

In [11]:
def DrawGraph_draw(encoding,coord):
    r"""
    returns a randomized plot of the graph defined by the encoding
    
    INPUT: 
     
     - ''encoding'' - a vector of the form $[m,n,s,e_1,e_2,...,e_2n]$
     
     - ''delta'' - a positive scalar
     
     - ''min_len'' - a positive scalar
     
     OUTPUT:
     
     - a graphics plot 
         
    """
    
    m=encoding[0]
    n=encoding[1]
    
    
    coordinates=coord.rows()
    points=point2d(coord,size=50)
    plot_list=[points]
    
    for i in range(n):
        source_vertice=coordinates[m+i]
        
        left_vertice_label=encoding[2*i+3]
        right_vertice_label=encoding[2*i+4]
        
        left_vertice=coordinates[left_vertice_label]
        right_vertice=coordinates[right_vertice_label]
        
        left_edge=arrow2d(source_vertice,left_vertice)
        right_edge=arrow2d(source_vertice,right_vertice)
        
        plot_list.append(left_edge)
        plot_list.append(right_edge)
        
        plot=points
    for i in range(len(plot_list)-1):
        plot+=plot_list[i+1]
    
    
    return plot

In [12]:
def intersection_test(p_1,p_2,q_1,q_2):
    r"""
    returns a value True or False indicating whether the line segments defined by the points intersect

    INPUT:

    - ''p_1'' - a vector indicating the source point of the first line segment

    - ''p_2'' - a vector indicating the end point of the first line segment

    - ''q_1'' - a vector indicating the source point of the second line segment

    - ''q_2'' - a vector indicating the end point of the second line segment

    OUTPUT:

    - a value True or False, depending on whether the line segments intersect

    EXAMPLES:
        sage: intersection_test(vector([1,1]),vector([4,4]),vector([3,0]),vector([1,4]))
        True
        sage: intersection_test(vector([1,0]),vector([3,3]),vector([5,2]),vector([3,5]))
        False
    """
    
    value=False
    #computing r
    r=zero_vector(RR,2)
    r[0]=p_2[0]-p_1[0]
    r[1]=p_2[1]-p_1[1]

    #computing s
    s=zero_vector(RR,2)
    s[0]=q_2[0]-q_1[0]
    s[1]=q_2[1]-q_1[1]

    r_x_s=r[0]*s[1]-r[1]*s[0]
    if r_x_s!=0:
        q_min_p=q_1-p_1
        q_min_p_x_s=q_min_p[0]*s[1]-q_min_p[1]*s[0]
        t=q_min_p_x_s/r_x_s

        q_min_p_x_r=q_min_p[0]*r[1]-q_min_p[1]*r[0]
        u=q_min_p_x_r/r_x_s

        if 0<=t<1 and 0<=u<=1:
            value=True

    return value

In [13]:
def target_function(encoding,coordinates):
    r"""
    returns a score denoting how 'beautiful' the drawing of the graph is
    
    INPUT:
    
    - ''encoding'' - a vector of the form $[m,n,s,e_1,e_2,...,e_2n]$
    
    - '' coordinates'' - an (n+m) x 2 matrix containing coordinates in the form [x_1,y_1;...;x_n+m,y_n+m]
    
    OUTPUT:
    
    - a scalar
    
    EXAMPLES:
    
    """
    
    #the parameters
    L_0 = 10
    c_se=1
    c_le=1
    alpha=1
    beta=1
    gamma=1
    epsilon=1
    
    #getting the prerequisites from the data
    m=encoding[0]
    n=encoding[1]
    
    #vertices_sink is an n x 4 matrix of which the first two columns give the coordinates for the left vertice with\
    #which the source is connected and the column 2 and 3 give the coordinates for the right vertice
    #edges_length is an n x 2 matrix of which the first column gives the lenght of the left edge of the source vertex\
    #and the second column gives the length of the right edge
    #edges_begin_end is a 2n x 4 matrix containing the begin and end point of each edges in the rows
    vertices_sink = zero_matrix(RR,n,4)
    edges_length=zero_matrix(RR,n,2)
    edges_begin_end=zero_matrix(RR,2*n,4)

    for i in range(n):
        e_L=encoding[3+2*i]
        e_R=encoding[4+2*i]
        vertices_sink[i,0:2]=coordinates[e_L,:]
        vertices_sink[i,2:4]=coordinates[e_R,:]
        
        coord_source_x=coordinates[m+i,0]
        coord_source_y=coordinates[m+i,1]
        edges_begin_end[[2*i,2*i+1],0]=coord_source_x
        edges_begin_end[[2*i,2*i+1],1]=coord_source_y

        coord_sink_L_x=coordinates[e_L,0]
        coord_sink_L_y=coordinates[e_L,1]
        edges_begin_end[2*i,2]=coord_sink_L_x
        edges_begin_end[2*i,3]=coord_sink_L_y


        coord_sink_R_x=coordinates[e_R,0]
        coord_sink_R_y=coordinates[e_R,1]
        edges_begin_end[2*i+1,2]=coord_sink_R_x
        edges_begin_end[2*i+1,3]=coord_sink_R_y
        
        delta_x_L=coord_source_x-coord_sink_L_x
        delta_y_L=coord_source_y-coord_sink_L_y
        
        delta_x_R=coord_source_x-coord_sink_R_x
        delta_y_R=coord_source_y-coord_sink_R_y
        
        edges_length[i,0] = sqrt(delta_x_L^2+delta_y_L^2)
        edges_length[i,1] = sqrt(delta_x_R^2+delta_y_R^2)
        

        
    
    # short edges
    short_edges_score=0
    #long edges
    long_edges_score = 0
    for i in range(n):
        length_L=edges_length[i,0]
        length_R=edges_length[i,1]
        
        left_short_edges_score = c_se*(length_L/L_0)^alpha
        right_short_edges_score =c_se*(length_R/L_0)^alpha
        
        left_long_edges_score = c_le*(L_0/length_L)^beta
        right_long_edges_score = c_le*(L_0/length_R)^beta
        
        short_edges_score += left_short_edges_score+right_short_edges_score
        long_edges_score += left_long_edges_score+right_long_edges_score
    
    #intersections
    intersections_score = 0
    num_of_intersections=0

    for i in range(2*n):
        p_0=vector([edges_begin_end[i,0],edges_begin_end[i,1]])
        p_1=vector([edges_begin_end[i,2],edges_begin_end[i,3]])

        for j in range(2*n):
            if i == j:
                continue
            q_0=vector([edges_begin_end[j,0],edges_begin_end[j,1]])
            q_1=vector([edges_begin_end[j,2],edges_begin_end[j,3]])

            if p_0==q_0 or p_0 == q_1:
                continue
            if p_1 == q_0 or p_1 == q_1:
                continue
            
            value=intersection_test(p_0,p_1,q_0,q_1)
            if value == True:
                num_of_intersections+=1
    if num_of_intersections !=0:
        intersections_score = num_of_intersections^gamma*RDF(log(num_of_intersections))^epsilon
    
    score=short_edges_score+long_edges_score+intersections_score
    
    return score

# Example Run
The cell below gives an example run for the algorithm.

In [14]:
# We first need to chose an encoding, we do this by hand in this case
# This encoding has 2 inner nodes, so we will get an output of 2 sets of x,y coordinates
encoding = vector([2,2,1,0,3,1,2])

# delta is the distance we want between our sink nodes, the nodes that lie on the x-axis
delta = 50

# Run the algorithm
coordinates = DrawGraph_algorithm(encoding,delta)

print("A possible set of coordinates is: ")
print(coordinates)
print("The first row gives the coordinates (x_1,y_1), the second row the coordinates (x_2,y_2)")

A possible set of coordinates is: 
[ 42.8571428571429 -28.5714285714286]
[ 42.8571428571429 -28.5714285714286]
The first row gives the coordinates (x_1,y_1), the second row the coordinates (x_2,y_2)


The cell below gives an example for how the target function can give a value to a set of coordinates:

In [15]:
value = target_function(encoding,coordinates)
print("The value is: " + value)

IndexError: matrix index out of range